# RNA Pre-Trained Model

In [1]:
 #Import All Libraries Here
import pandas as pd
from sklearn.metrics import accuracy_score ,  roc_curve, auc , classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split , RandomizedSearchCV


from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import RandomOverSampler


from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import optuna
from torch.utils.data import WeightedRandomSampler

import gensim

# PyTorch Import

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Record Constants
INPUT_TRAIN_IN = "../../../data/train_in.csv"
INPUT_TRAIN_OUT = "../../../data/train_out.csv"  
INPUT_TEST_IN = "../../../data/test_in.csv"
INPUT_TEST_OUT = "../../../data/test_out.csv"
INPUT_VALIDATION_IN = "../../../data/valid_in_nucleo.csv"
INPUT_VALIDATION_OUT  = "../../../data/valid_out.csv"

TARGET_MODEL_PATH = '../../webapp/model_files'


# 1 - One Hot Encoding with Pytorch in build Emnedding 
# 2 - K-mers with Word2Vec
ENCODING_METHOD = 1


# 1 - LSTM with Cross Entropy 
MODEL = 1


FRAMEWORK = "PYTORCH"

# Startegy to Crop Sequene
# MID - Modification is present at Mid of cropped Sequence 
# END - Modification is present at End of cropepd Sequence 
CROP_STRATEGY = 'END'

# Y Category Encoding Method
# LABEL or ONE_HOT 
TARGET_ENCODING = 'LABEL'



c:\Users\shashi.vish\Python Environment\RNA_ModX\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Read X Varaibles and Y Varaibles

x_train_raw =  pd.read_csv(INPUT_TRAIN_IN, header=None , skiprows=1 )
y_train_raw =  pd.read_csv(INPUT_TRAIN_OUT, header=None , skiprows=1 )

x_test_raw =  pd.read_csv(INPUT_TEST_IN, header=None , skiprows=1 )
y_test_raw =  pd.read_csv(INPUT_TEST_OUT, header=None , skiprows=1)

x_valid_raw =  pd.read_csv(INPUT_VALIDATION_IN, header=None , skiprows=1 )
y_valid_raw =  pd.read_csv(INPUT_VALIDATION_OUT, header=None , skiprows=1 )



In [3]:
print(f"Train Shape of X : {x_train_raw.shape} and Tranin Shape of Y : {x_train_raw.shape}")
print(f"Test Shape of X : {x_test_raw.shape} and Test Shape of Y : {y_test_raw.shape}")
print(f"Validation Shape of X : {x_valid_raw.shape} and Validation Shape of Y : {y_valid_raw.shape}")

Train Shape of X : (304661, 1001) and Tranin Shape of Y : (304661, 1001)
Test Shape of X : (1200, 1001) and Test Shape of Y : (1200, 12)
Validation Shape of X : (3599, 1001) and Validation Shape of Y : (3599, 12)


### Encode with One Hot Encoding

In [4]:
number_of_unique_kmers = set()
def encode_seq(kmer_token):

    # A 1 0 0 0
    # C 0 1 0 0
    # T/U 0 0 0 1
    # G 0 0 1 0
    # N 0 0 0 0

    encoding_dict = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'U': [0, 0, 0, 1],
        'N': [0, 0, 0, 0],
    }

    encoded_sequence = []
    number_of_unique_kmers.add(kmer_token)
    for  base in kmer_token:
        encoded_sequence.append(encoding_dict[base])
    return np.array(encoded_sequence).flatten()

def applyOneHotEncoding(tokenized_sequences):
    encoded_sequences = []
    for seq in tokenized_sequences:
        encoded_sequences.append(encode_seq(seq)) 
 
    return np.array(encoded_sequences).flatten()

def encode_with_one_hot_encoding(x):
    concatenated_column= x.apply(lambda row: ''.join(map(str, row)), axis=1)
    df_result = x.assign(Sequence=concatenated_column)
    tokenized_sequences =  df_result['Sequence'].apply(applyOneHotEncoding).tolist()
    return tokenized_sequences


In [5]:
X_train = torch.tensor(encode_with_one_hot_encoding(x_train_raw) , dtype=torch.long)
X_test = torch.tensor(encode_with_one_hot_encoding(x_test_raw) , dtype=torch.long)
X_valid = torch.tensor(encode_with_one_hot_encoding(x_valid_raw) , dtype=torch.long)

C:\Users\shashi.vish\AppData\Local\Temp\ipykernel_32188\3010200440.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  X_train = torch.tensor(encode_with_one_hot_encoding(x_train_raw) , dtype=torch.long)


In [6]:
RMs = ['hAm','hCm','hGm','hTm','hm1A','hm5C','hm5U','hm6A','hm6Am','hm7G','hPsi','Atol','NonMoD']
RMEncoding = [12,1,2,3,4,5,6,7,8,9,10,11,0]

def convert_y_to_original_labels(row):
    label = ""
    for index , n in enumerate(row.tolist()) :
        if n == 1 :
            label = RMs[index]
    if label == '':
        return 'NonMoD'
    return label

def get_original_y_lables( y_data ):
    # Convert One Hot Encoded Y to to Original Labels 
    y_original_labels = y_data.apply(convert_y_to_original_labels,axis=1)
    return y_original_labels  


In [7]:
def encode_target(y_data):
    # Write Customer Lable Encoder . This is required since we have train and test alreday splitted. Always creating a new instanc of label encoder will change encoding.
    y_encoded = []
    for y in y_data:
        index = RMs.index(y)
        encoding =  RMEncoding[index]
        y_encoded.append(encoding)
    return y_encoded

y_train_original =  get_original_y_lables(y_train_raw)
y_test_original =  get_original_y_lables(y_test_raw )
y_valid_original =  get_original_y_lables(y_valid_raw )

y_train = encode_target(y_train_original)
y_test = encode_target(y_test_original)
y_valid = encode_target(y_valid_original)

y_train = torch.tensor(y_train , dtype=torch.long)
y_test = torch.tensor(y_test , dtype=torch.long)
y_valid = torch.tensor(y_valid , dtype=torch.long)
y_train

# Calculate class weights
class_weights = 1.0 / torch.bincount(y_train)
print(class_weights)
weights = class_weights[y_train]
sampler = WeightedRandomSampler(weights, len(weights))

tensor([6.5700e-06, 5.9595e-04, 7.8678e-04, 4.8709e-04, 6.1954e-05, 3.3256e-04,
        2.8604e-04, 1.5390e-05, 4.4504e-04, 1.1962e-03, 3.4048e-04, 1.9077e-05,
        7.1891e-04])


In [15]:
hyperparameter = {}

hyperparameter['INPUT_DIMENSION'] = 4 # For One Hot Encoding Input Dimension would be 4 as there only 4 unique nucleocide 
hyperparameter['HIDDEN_DIMENSION'] = 3
hyperparameter['NO_OF_LAYERS'] = 1
hyperparameter['BATCH_SIZE'] = 32
hyperparameter['OUTPUT_DIMENSION'] = 13
hyperparameter['EMBEDDING_DIMENSION'] = 7 # if you are using Word2Vec Encoding then this should be same as Word2Vec Embedding Dim 
hyperparameter['DROP_OUT'] = 0.5

In [9]:
class RNADataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

train_dataset = RNADataset(X_train, y_train)
test_dataset = RNADataset(X_test, y_test)
valid_dataset = RNADataset(X_valid, y_valid)


train_dataloader = DataLoader(train_dataset, batch_size=hyperparameter['BATCH_SIZE'], sampler=sampler)
test_dataloader = DataLoader(test_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)

In [16]:
class RNATransferLearning(nn.Module):
    def __init__(self, input_dim,embedding_dim, hidden_dim,num_layers, output_dim , dropout=0.5):
        super(RNATransferLearning, self).__init__()

        #Pytroch Embedding 
        self.embedding = nn.Embedding(input_dim,embedding_dim)

        #Bi-Directional LSTM Model
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,num_layers, dropout= dropout, batch_first=True )

        #Fully Connected Layer   
        self.fc = nn.Linear(hidden_dim, output_dim) 

        # Drop out layer for Overfitting 
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):

        x  = x.long()

        x_embeded =  self.embedding(x)
        
        lstm_out, (h,c) = self.lstm(x_embeded)

        # Pass it to drop out layer 
        output = self.dropout(h[-1])
        
        # Finally pass it to fully connected layer.
        out = self.fc(output)
        
        return out # Sequeez to Single dimension for loss calculation
    

def validate_model(model, test_dataloader , device ,loss_function):
    model.eval()
    running_loss = 0.0
    class_correct = [0] * hyperparameter['OUTPUT_DIMENSION'] 
    class_total = [0] * hyperparameter['OUTPUT_DIMENSION'] 
    total = 0
    correct = 0

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.long().to(device)

            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
            
    validation_loss = running_loss / len(test_dataloader)
    validation_accuracies = {f"Class {i}": 100 * class_correct[i] / class_total[i] for i in range(hyperparameter['OUTPUT_DIMENSION'] )}
    overall_accuracy =  correct / total
    return validation_loss , validation_accuracies , overall_accuracy


def train_model(model, train_dataloader, test_dataloader, device, epochs, optimizer, loss_function):
    best_val_loss = float('inf')
    no_improvement_count = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for i, (inputs, labels) in enumerate(train_dataloader):
            inputs = inputs.to(device)
            labels = labels.long().to(device)

            optimizer.zero_grad()
            outputs = model(inputs) 
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_dataloader)
        val_loss, val_accuracies, overall_accuracy = validate_model(model, test_dataloader, device, loss_function)
        
        print(f"Epoch {epoch + 1}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Test Accuracy: {overall_accuracy:.4f}")
        for i in range(hyperparameter['OUTPUT_DIMENSION']):
            print(f"Class {i} Accuracy: {val_accuracies[f'Class {i}']:.2f}%")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improvement_count = 0
        else:
            no_improvement_count += 1
            if no_improvement_count == 3:
                print("No improvement in validation loss for 3 epochs. Training stopped.")
                break


model = RNATransferLearning(input_dim=hyperparameter['INPUT_DIMENSION'],
                            embedding_dim=hyperparameter['EMBEDDING_DIMENSION'], 
                            hidden_dim=hyperparameter['HIDDEN_DIMENSION'] , 
                            num_layers = hyperparameter['NO_OF_LAYERS'],
                            output_dim=hyperparameter['OUTPUT_DIMENSION'],
                            dropout=hyperparameter['DROP_OUT'] )


loss_function = nn.CrossEntropyLoss()  ## MSELoss of Regression problem  # BCELoss for binary classification
optimizer = optim.Adam(model.parameters() ,  lr=0.01)

# Number of Parameters for Model
total_parameters = []
for p in model.parameters():
    total_parameters.append(p.numel())

print(f"Total Number of Parameters for Model Training : { sum(total_parameters)} " )

# Train the model
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Model Parameters  : " , hyperparameter)

# Train Model with configured Parameter
train_model(model, train_dataloader ,test_dataloader, device ,num_epochs,optimizer,loss_function)



Total Number of Parameters for Model Training : 224 
Model Parameters  :  {'INPUT_DIMENSION': 4, 'HIDDEN_DIMENSION': 3, 'NO_OF_LAYERS': 1, 'BATCH_SIZE': 32, 'OUTPUT_DIMENSION': 13, 'EMBEDDING_DIMENSION': 7, 'DROP_OUT': 0.5}
Epoch 1, Train Loss: 2.3536, Val Loss: 2.4321, Test Accuracy: 0.0833
Class 0 Accuracy: 0.00%
Class 1 Accuracy: 0.00%
Class 2 Accuracy: 0.00%
Class 3 Accuracy: 100.00%
Class 4 Accuracy: 0.00%
Class 5 Accuracy: 0.00%
Class 6 Accuracy: 0.00%
Class 7 Accuracy: 0.00%
Class 8 Accuracy: 0.00%
Class 9 Accuracy: 0.00%
Class 10 Accuracy: 0.00%
Class 11 Accuracy: 100.00%
Class 12 Accuracy: 0.00%
Epoch 2, Train Loss: 2.3447, Val Loss: 2.5833, Test Accuracy: 0.0833
Class 0 Accuracy: 0.00%
Class 1 Accuracy: 0.00%
Class 2 Accuracy: 0.00%
Class 3 Accuracy: 0.00%
Class 4 Accuracy: 0.00%
Class 5 Accuracy: 100.00%
Class 6 Accuracy: 0.00%
Class 7 Accuracy: 0.00%
Class 8 Accuracy: 0.00%
Class 9 Accuracy: 0.00%
Class 10 Accuracy: 0.00%
Class 11 Accuracy: 100.00%
Class 12 Accuracy: 0.00%


In [11]:
model_name = 'rna_pre_trained_model.pt'
torch.save(model.state_dict(), model_name)

In [12]:
import optuna

hyperparameter = {}

hyperparameter['INPUT_DIMENSION'] = 4 
hyperparameter['OUTPUT_DIMENSION'] = 13

def objective(trial):

    embedding_dim = trial.suggest_int("embedding_dim", 16, 300)
    hidden_dim = trial.suggest_int("hidden_dim", 32, 512)
    num_layers = trial.suggest_int("num_layers", 1, 4)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", [256, 512, 1024])

    model = RNATransferLearning(
        input_dim=hyperparameter['INPUT_DIMENSION'],
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        output_dim=hyperparameter['OUTPUT_DIMENSION'],
        dropout=dropout
    )

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    total_parameters = sum(p.numel() for p in model.parameters())
    print(f"Total Number of Parameters for Model Training: {total_parameters}")

    num_epochs = 100
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    print(f"hyperparameter['INPUT_DIMENSION'] : {hyperparameter['INPUT_DIMENSION']},hyperparameter['OUTPUT_DIMENSION'] : {hyperparameter['OUTPUT_DIMENSION']},embedding_dim : {embedding_dim},hidden_dim : {hidden_dim},num_layers :{num_layers},dropout : {dropout},learning_rate : {learning_rate},batch_size : {batch_size}")

    train_model(model, train_dataloader ,test_dataloader, device ,num_epochs,optimizer,loss_function)
    val_loss, val_accuracies, overall_accuracy = validate_model(model, test_dataloader, device, loss_function)

    return val_loss


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value:.4f}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2023-06-02 18:24:29,081] A new study created in memory with name: no-name-1751e192-ec03-4eef-99ba-deb823c7c881
C:\Users\shashi.vish\AppData\Local\Temp\ipykernel_32188\2422976769.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)


Total Number of Parameters for Model Training: 4214602
hyperparameter['INPUT_DIMENSION'] : 4,hyperparameter['OUTPUT_DIMENSION'] : 13,embedding_dim : 202,hidden_dim : 373,num_layers :4,dropout : 0.4918563535970837,learning_rate : 0.006729541321466909,batch_size : 1024


[W 2023-06-02 18:26:12,397] Trial 0 failed with parameters: {'embedding_dim': 202, 'hidden_dim': 373, 'num_layers': 4, 'dropout': 0.4918563535970837, 'learning_rate': 0.006729541321466909, 'batch_size': 1024} because of the following error: RuntimeError('[enforce fail at ..\\c10\\core\\impl\\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 118432612352 bytes.').
Traceback (most recent call last):
  File "c:\Users\shashi.vish\Python Environment\RNA_ModX\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\shashi.vish\AppData\Local\Temp\ipykernel_32188\2422976769.py", line 42, in objective
    train_model(model, train_dataloader ,test_dataloader, device ,num_epochs,optimizer,loss_function)
  File "C:\Users\shashi.vish\AppData\Local\Temp\ipykernel_32188\281444117.py", line 82, in train_model
    outputs = model(inputs)
  File "c:\Users\shashi.vish\Python Environment\RNA_ModX\lib\site-p

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 118432612352 bytes.

In [ ]:
window_size = 1000
target_size = 1
model_name = "model_feature_windoe_"+str(window_size)+"_target_"+str(target_size)
torch.save(model.state_dict, model_name)